## This example demonstrates XCSF hyperparameter tuning

In [1]:
import json

import numpy as np
import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_param_importances
from sklearn.compose import TransformedTargetRegressor
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from optuna.integration import OptunaSearchCV
from optuna.distributions import FloatDistribution

import xcsf

RANDOM_STATE: int = 1

np.random.seed(RANDOM_STATE)

### Load training and test data
Loads the kin8nm dataset from: https://www.openml.org/d/189

In [2]:
data = fetch_openml(data_id=189, as_frame=True, parser="auto")

data.frame.head()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
0,-0.015119,0.360741,0.469398,1.309675,0.988024,-0.025493,0.664071,0.062763,0.536524
1,0.360478,-0.301395,0.629183,-1.440146,-0.741637,-1.196749,-1.038444,-0.717461,0.308014
2,1.563238,-1.294753,0.078987,1.432937,1.149136,-1.292140,1.562988,-0.937731,0.518900
3,0.199485,0.901157,-1.356304,-0.080525,-0.976628,0.829894,-0.855649,0.930630,0.494151
4,0.659737,0.120552,-0.008756,0.648839,0.626832,-0.646539,1.318074,-0.899172,0.470218


In [3]:
data.frame.describe()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
count,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000
mean,0.009351,-0.007233,0.022792,-0.012945,0.007886,-0.009698,0.008402,0.001227,0.714283
std,0.904867,0.901496,0.900765,0.899186,0.906874,0.913243,0.909724,0.906098,0.263607
min,-1.570611,-1.570681,-1.570669,-1.570295,-1.570664,-1.570254,-1.570643,-1.568904,0.040165
25%,-0.770328,-0.778319,-0.749718,-0.780602,-0.770492,-0.802084,-0.776870,-0.777432,0.523584
50%,-0.002010,-0.015152,0.041346,-0.013617,0.009702,-0.003575,0.001338,0.002007,0.708822
75%,0.791179,0.779145,0.792391,0.760165,0.791203,0.781849,0.798014,0.780927,0.901354
max,1.570753,1.570657,1.570736,1.570169,1.570599,1.570597,1.570522,1.570237,1.458521


In [4]:
# numpy
X_orig = np.asarray(data.data, dtype=np.float64)
y_orig = np.asarray(data.target, dtype=np.float64)

# normalise inputs (zero mean and unit variance)
feature_scaler = StandardScaler()
X = feature_scaler.fit_transform(X_orig)

# reshape outputs into 2D arrays
if y_orig.ndim == 1:
    y_orig = y_orig.reshape(-1, 1)

# normalise outputs (zero mean and unit variance)
output_scaler = StandardScaler()
y = output_scaler.fit_transform(y_orig)

# split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE
)

# 10% of training for validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=RANDOM_STATE
)

print(f"X_train shape = {np.shape(X_train)}")
print(f"y_train shape = {np.shape(y_train)}")
print(f"X_val shape = {np.shape(X_val)}")
print(f"y_val shape = {np.shape(y_val)}")
print(f"X_test shape = {np.shape(X_test)}")
print(f"y_test shape = {np.shape(y_test)}")

X_DIM: int = np.shape(X_train)[1]
Y_DIM: int = np.shape(y_train)[1]

X_train shape = (6634, 8)
y_train shape = (6634, 1)
X_val shape = (738, 8)
y_val shape = (738, 1)
X_test shape = (820, 8)
y_test shape = (820, 1)


### Initialise XCSF

In [5]:
# Due to the way XCSF sets parameters for conditions, actions, and predictions,
# it is necessary to set the desired parameters in a wrapper function...


def get_prediction(n_init: int) -> dict:
    """Returns parameters for initialising neural network predictions."""
    return {
        "type": "neural",
        "args": {
            "layer_0": {
                "type": "connected",
                "activation": "selu",
                "n_init": n_init,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
            "layer_1": {
                "type": "connected",
                "activation": "selu",
                "n_init": Y_DIM,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
        },
    }


def get_model() -> xcsf.XCS:
    """Returns a new XCS with baseline parameters."""
    return xcsf.XCS(
        x_dim=X_DIM,
        y_dim=Y_DIM,
        n_actions=1,
        omp_num_threads=12,
        random_state=RANDOM_STATE,
        pop_init=True,
        max_trials=200000,
        perf_trials=5000,
        pop_size=500,
        loss_func="mse",
        set_subsumption=False,
        theta_sub=100,
        e0=0.1,
        alpha=1,
        nu=20,
        beta=0.1,
        delta=0.1,
        theta_del=50,
        init_fitness=0.01,
        init_error=0,
        m_probation=10000,
        stateful=True,
        compaction=False,
        ea={
            "select_type": "roulette",
            "theta_ea": 50,
            "lambda": 2,
            "p_crossover": 0.8,
            "err_reduc": 1,
            "fit_reduc": 0.1,
            "subsumption": False,
            "pred_reset": False,
        },
        condition={
            "type": "tree_gp",
            "args": {
                "min_constant": 0,
                "max_constant": 1,
                "n_constants": 100,
                "init_depth": 5,
                "max_len": 10000,
            },
        },
        prediction=get_prediction(40),
    )

In [6]:
def get_callback():
    """Returns a new callback for early stoppping."""
    return xcsf.EarlyStoppingCallback(
        # note: PERF_TRIALS is considered an "epoch" for callbacks
        monitor="val",  # which metric to monitor: {"train", "val"}
        patience=20000,  # trials with no improvement after which training will be stopped
        restore_best=True,  # whether to make checkpoints and restore best population
        min_delta=0,  # minimum change to qualify as an improvement
        start_from=0,  # trials to wait before starting to monitor improvement
        verbose=True,  # whether to display when checkpoints are made
    )

### Sklearn pipeline

Note: sklearn pipelines do not allow the use of callback functions.

In [7]:
# split (original data) into training and test sets
X_tr, X_te, y_tr, y_te = train_test_split(
    X_orig, y_orig, test_size=0.1, random_state=RANDOM_STATE
)

model = get_model()
model.set_params(max_trials=50000)  # speed this demo up

pipe = make_pipeline(
    StandardScaler(),
    TransformedTargetRegressor(regressor=model, transformer=StandardScaler()),
)

pipe.fit(X_tr, y_tr)

00:00:01.161 trials=0 train=1.86722 pset=500.0 mset=238.9 mfrac=0.65
00:00:01.246 trials=5000 train=0.22114 pset=500.0 mset=223.5 mfrac=0.82
00:00:01.127 trials=10000 train=0.18990 pset=500.0 mset=366.9 mfrac=1.00
00:00:01.630 trials=15000 train=0.24111 pset=500.0 mset=405.6 mfrac=0.98
00:00:01.623 trials=20000 train=0.14901 pset=500.0 mset=378.3 mfrac=0.96
00:00:01.674 trials=25000 train=0.15173 pset=500.0 mset=414.2 mfrac=0.94
00:00:01.637 trials=30000 train=0.13972 pset=500.0 mset=406.4 mfrac=0.37
00:00:01.654 trials=35000 train=0.12362 pset=500.0 mset=394.7 mfrac=0.68
00:00:01.617 trials=40000 train=0.11709 pset=500.0 mset=390.1 mfrac=0.87
00:00:01.724 trials=45000 train=0.11693 pset=500.0 mset=384.0 mfrac=0.99
00:00:15.093


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('transformedtargetregressor',
                 TransformedTargetRegressor(regressor=<xcsf.xcsf.XCS object at 0x7f7823e5e730>,
                                            transformer=StandardScaler()))])

In [8]:
train_score = pipe.score(X_tr, y_tr)
test_score = pipe.score(X_te, y_te)

print(f"train = {train_score}")
print(f"test = {test_score}")

train = 0.8870088321372127
test = 0.8812029081717271


### Sklearn grid search

In [9]:
# General parameters can be searched in the usual way

parameters = dict(beta=[0.1, 0.5])

# EA parameters require specifying a dict,
# but individual values can still be set because the other values are not changed.

parameters = dict(ea=[{"lambda": 2}, {"lambda": 10}])

# However, for actions, conditions, and predictions, the WHOLE dict must be specified
# for each value to try in the search. See Optuna below.

model = get_model()
model.set_params(max_trials=25000)  # speed this demo up

grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring="neg_mean_squared_error",
    cv=3,
)

grid_search.fit(X_train, y_train)

results = grid_search.cv_results_

for mean_score, std_score, params in zip(
    results["mean_test_score"], results["std_test_score"], results["params"]
):
    print("Mean Score:", -mean_score)
    print("Standard Deviation:", std_score)
    print("Parameters:", params)
    print("------------------------")

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", -grid_search.best_score_)

00:00:00.880 trials=0 train=1.93661 pset=500.0 mset=237.9 mfrac=0.62
00:00:00.872 trials=5000 train=0.26937 pset=500.0 mset=230.3 mfrac=0.82
00:00:01.314 trials=10000 train=0.20115 pset=500.0 mset=369.0 mfrac=0.62
00:00:02.220 trials=15000 train=0.16725 pset=500.0 mset=403.1 mfrac=0.87
00:00:01.882 trials=20000 train=0.17910 pset=500.0 mset=373.2 mfrac=0.81
00:00:07.168
00:00:00.980 trials=0 train=2.61311 pset=500.0 mset=236.5 mfrac=1.00
00:00:00.955 trials=5000 train=0.22297 pset=500.0 mset=213.3 mfrac=0.75
00:00:01.144 trials=10000 train=0.23937 pset=500.0 mset=396.2 mfrac=0.97
00:00:02.264 trials=15000 train=0.16062 pset=500.0 mset=413.3 mfrac=0.65
00:00:02.155 trials=20000 train=0.14551 pset=500.0 mset=407.7 mfrac=0.66
00:00:07.498
00:00:01.264 trials=0 train=2.04997 pset=500.0 mset=238.0 mfrac=0.98
00:00:01.185 trials=5000 train=0.22114 pset=500.0 mset=217.2 mfrac=0.71
00:00:01.417 trials=10000 train=0.16402 pset=500.0 mset=406.6 mfrac=0.50
00:00:02.195 trials=15000 train=0.14423 

### Optuna

In [10]:
def objective(trial):
    """Measure a new hyperparameter combination."""
    # get new parameters to try
    e0: float = trial.suggest_float("e0", 0.08, 0.12, step=0.01)
    beta: float = trial.suggest_float("beta", 0.01, 0.19, step=0.02)
    n_init: int = trial.suggest_int("n_init", 1, 101, step=10)
    # get a new XCSF and set params - predictions must include the whole dict
    prediction: dict = get_prediction(n_init=n_init)
    model = get_model()
    model.set_params(e0=e0, beta=beta, prediction=prediction)
    # get a new early stopping callback
    callback = get_callback()
    # fit XCSF
    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        callbacks=[callback],
        verbose=True,
    )
    # optimise validation MSE
    return model.score(X_val, y_val)


study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=RANDOM_STATE))

[I 2023-07-19 13:41:14,484] A new study created in memory with name: no-name-1caa78c5-e8cc-4c34-8210-6fffc19d7bda


In [ ]:
study.optimize(objective, n_trials=10)

00:00:00.650 trials=0 train=0.66547 val=0.71319 pset=500.0 mset=278.6 mfrac=0.06
checkpoint: 0.71319 error at 0 trials
00:00:00.679 trials=5000 train=0.59617 val=0.60174 pset=500.0 mset=307.7 mfrac=0.86
checkpoint: 0.60174 error at 5000 trials
00:00:00.799 trials=10000 train=0.60200 val=0.60767 pset=500.0 mset=391.6 mfrac=0.31
00:00:00.839 trials=15000 train=0.57202 val=0.58019 pset=500.0 mset=357.1 mfrac=0.01
checkpoint: 0.58019 error at 15000 trials
00:00:00.967 trials=20000 train=0.56980 val=0.60708 pset=500.0 mset=342.9 mfrac=0.00
00:00:00.959 trials=25000 train=0.57959 val=0.60004 pset=500.0 mset=330.3 mfrac=0.22
00:00:01.461 trials=30000 train=0.59414 val=0.57761 pset=500.0 mset=315.5 mfrac=0.00
checkpoint: 0.57761 error at 30000 trials
00:00:01.070 trials=35000 train=0.57111 val=0.60486 pset=500.0 mset=285.3 mfrac=0.11
00:00:01.029 trials=40000 train=0.57850 val=0.62303 pset=500.0 mset=273.4 mfrac=0.02


[I 2023-07-19 13:41:27,110] Trial 0 finished with value: 0.5776050574797028 and parameters: {'e0': 0.1, 'beta': 0.15000000000000002, 'n_init': 1}. Best is trial 0 with value: 0.5776050574797028.


00:00:01.084 trials=45000 train=0.57214 val=0.60767 pset=500.0 mset=254.4 mfrac=0.00
00:00:00.912 trials=50000 train=0.57227 val=0.62378 pset=500.0 mset=244.8 mfrac=0.00
00:00:02.041 trials=55000 train=0.56789 val=0.59860 pset=500.0 mset=224.2 mfrac=0.13
restoring system from trial 30000 with error=0.57761
00:00:12.490
00:00:00.978 trials=0 train=2.94642 val=8.36705 pset=500.0 mset=214.5 mfrac=1.00
checkpoint: 8.36705 error at 0 trials
00:00:00.869 trials=5000 train=0.41665 val=0.29036 pset=500.0 mset=217.7 mfrac=1.00
checkpoint: 0.29036 error at 5000 trials
00:00:01.159 trials=10000 train=0.30444 val=0.28284 pset=500.0 mset=387.7 mfrac=1.00
checkpoint: 0.28284 error at 10000 trials
00:00:01.499 trials=15000 train=0.29390 val=2.57404 pset=500.0 mset=428.5 mfrac=1.00
00:00:01.537 trials=20000 train=0.24860 val=0.21924 pset=500.0 mset=435.0 mfrac=1.00
checkpoint: 0.21924 error at 20000 trials
00:00:01.565 trials=25000 train=0.23821 val=0.28609 pset=500.0 mset=447.0 mfrac=1.00
00:00:01.61

[I 2023-07-19 13:42:04,646] Trial 1 finished with value: 0.163246205441045 and parameters: {'e0': 0.09, 'beta': 0.03, 'n_init': 11}. Best is trial 1 with value: 0.163246205441045.


00:00:00.848 trials=105000 train=0.16832 val=0.16348 pset=500.0 mset=171.1 mfrac=0.66
restoring system from trial 80000 with error=0.16325
00:00:37.365
00:00:01.330 trials=0 train=6.62380 val=0.25126 pset=500.0 mset=256.2 mfrac=1.00
checkpoint: 0.25126 error at 0 trials
00:00:01.779 trials=5000 train=0.33433 val=0.23027 pset=500.0 mset=243.1 mfrac=0.50
checkpoint: 0.23027 error at 5000 trials
00:00:01.680 trials=10000 train=0.19652 val=0.19619 pset=500.0 mset=382.4 mfrac=1.00
checkpoint: 0.19619 error at 10000 trials
00:00:01.986 trials=15000 train=0.63862 val=0.18836 pset=500.0 mset=366.2 mfrac=0.13
checkpoint: 0.18836 error at 15000 trials
00:00:01.917 trials=20000 train=0.18317 val=0.19169 pset=500.0 mset=348.7 mfrac=0.25
00:00:02.010 trials=25000 train=0.19676 val=0.17326 pset=500.0 mset=353.3 mfrac=0.69
checkpoint: 0.17326 error at 25000 trials
00:00:02.630 trials=30000 train=0.18676 val=0.17340 pset=500.0 mset=362.8 mfrac=0.73
00:00:02.302 trials=35000 train=0.15162 val=0.16378 p

...

00:00:01.474 trials=195000 train=0.12211 val=0.11370 pset=500.0 mset=202.6 mfrac=1.00
restoring system from trial 175000 with error=0.11322
00:01:17.624
00:00:02.169 trials=0 train=3.14229 val=0.30290 pset=500.0 mset=254.7 mfrac=1.00
checkpoint: 0.30290 error at 0 trials
00:00:02.268 trials=5000 train=0.24855 val=0.22175 pset=500.0 mset=257.8 mfrac=1.00
checkpoint: 0.22175 error at 5000 trials
00:00:03.091 trials=10000 train=0.18064 val=0.18820 pset=500.0 mset=431.8 mfrac=1.00
checkpoint: 0.18820 error at 10000 trials
00:00:04.849 trials=15000 train=0.15456 val=0.16433 pset=500.0 mset=456.5 mfrac=1.00
checkpoint: 0.16433 error at 15000 trials
00:00:04.069 trials=20000 train=0.13526 val=0.15504 pset=500.0 mset=456.0 mfrac=1.00
checkpoint: 0.15504 error at 20000 trials
00:00:03.930 trials=25000 train=0.12835 val=0.15627 pset=500.0 mset=436.2 mfrac=1.00
00:00:03.873 trials=30000 train=0.12078 val=0.14156 pset=500.0 mset=422.6 mfrac=0.74
checkpoint: 0.14156 error at 30000 trials
00:00:03.8

[I 2023-07-19 13:45:19,889] Trial 3 finished with value: 0.11626020791865813 and parameters: {'e0': 0.1, 'beta': 0.09, 'n_init': 71}. Best is trial 2 with value: 0.11322288526340023.


00:00:00.997 trials=0 train=0.73747 val=0.62591 pset=500.0 mset=292.0 mfrac=0.00
checkpoint: 0.62591 error at 0 trials
00:00:01.040 trials=5000 train=0.65052 val=0.63479 pset=500.0 mset=335.9 mfrac=1.00
00:00:01.614 trials=10000 train=0.60005 val=0.62159 pset=500.0 mset=416.3 mfrac=0.83
checkpoint: 0.62159 error at 10000 trials
00:00:01.131 trials=15000 train=0.60339 val=0.59337 pset=500.0 mset=422.8 mfrac=0.59
checkpoint: 0.59337 error at 15000 trials
00:00:01.150 trials=20000 train=0.57634 val=0.65908 pset=500.0 mset=433.6 mfrac=0.59
00:00:01.014 trials=25000 train=0.56291 val=0.58915 pset=500.0 mset=420.0 mfrac=0.00
checkpoint: 0.58915 error at 25000 trials
00:00:01.002 trials=30000 train=0.58598 val=0.58283 pset=500.0 mset=393.5 mfrac=0.59
checkpoint: 0.58283 error at 30000 trials
00:00:01.024 trials=35000 train=0.58693 val=0.62762 pset=500.0 mset=378.6 mfrac=0.00
00:00:01.125 trials=40000 train=0.60294 val=0.58588 pset=500.0 mset=364.2 mfrac=0.24
00:00:01.103 trials=45000 train=0.

[I 2023-07-19 13:45:57,958] Trial 4 finished with value: 0.5714247941602284 and parameters: {'e0': 0.09, 'beta': 0.17, 'n_init': 1}. Best is trial 2 with value: 0.11322288526340023.


00:00:07.613 trials=105000 train=0.56151 val=0.59728 pset=500.0 mset=151.2 mfrac=0.00
restoring system from trial 80000 with error=0.57142
00:00:37.907
00:00:01.944 trials=0 train=3.41403 val=0.47920 pset=500.0 mset=245.0 mfrac=1.00
checkpoint: 0.47920 error at 0 trials
00:00:02.403 trials=5000 train=0.26052 val=0.23796 pset=500.0 mset=266.2 mfrac=1.00
checkpoint: 0.23796 error at 5000 trials
00:00:02.577 trials=10000 train=0.18937 val=0.19717 pset=500.0 mset=448.2 mfrac=1.00
checkpoint: 0.19717 error at 10000 trials
00:00:03.661 trials=15000 train=0.15972 val=0.16586 pset=500.0 mset=465.7 mfrac=1.00
checkpoint: 0.16586 error at 15000 trials
00:00:04.516 trials=20000 train=0.13991 val=0.15290 pset=500.0 mset=470.0 mfrac=1.00
checkpoint: 0.15290 error at 20000 trials
00:00:03.564 trials=25000 train=0.13048 val=0.14554 pset=500.0 mset=477.0 mfrac=1.00
checkpoint: 0.14554 error at 25000 trials
00:00:04.252 trials=30000 train=0.12044 val=0.14314 pset=500.0 mset=480.0 mfrac=1.00
checkpoint:

In [ ]:
best_trial = study.best_trial
print(f"Best trial - Evaluation metric: {best_trial.value:.4f}")
print(f"Best trial - Hyperparameters: {best_trial.params}")

In [ ]:
plot_param_importances(study)